In [1]:
file = open('english_speeches_date_place_title_text.txt',encoding="utf8")

In [2]:
import spacy

In [3]:
nlp = spacy.load('en',disable = ['parser','tagger','ner'])

In [4]:
nlp.max_length = 1723200

In [5]:
def seperate_punch(doc_text):
    return[ token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n •·']

In [6]:
file = file.read()

In [7]:
tokens = seperate_punch(file)

In [8]:
#Making lists of 11 words
#We are going to set 10 as length of seed text and then going to predict 11th word
train_len = 10 + 1
text_seq = []
#Creates list of lists of 11 words each
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_seq.append(seq)

In [9]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)

In [11]:
sequences  = tokenizer.texts_to_sequences(text_seq)

In [12]:
#setting our vocaboulary size to total number of unique words we have
vocab_size = len(tokenizer.word_counts)

In [13]:
import numpy as np

In [14]:
#Converting to numpy array
sequences = np.array(sequences)

In [15]:
from keras.utils import to_categorical

In [16]:
#X is our feature label
#Taking first ten words form the array as our features
X = sequences[:,:-1]

In [17]:
#Taking last word of the array as our prediction 
y = sequences[:,-1]

In [18]:
#One-Hot-Encoding
#num_classes = vocab_size+1 , one extra for padding
y = to_categorical(y,num_classes=vocab_size+1)

In [19]:
#no of words in seed text
seq_ln = 10 

In [20]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [36]:
#Modeldefination
#Takes parameter vocab_size and length of seed text
#Using one embedding layer, two LSTM layers and two dense layers
#Printing summary and returning model
def create_model(vocab_size,seq_ln):
    model = Sequential()
    model.add(Embedding(vocab_size,seq_ln,input_length=seq_ln))
    model.add(LSTM(80,return_sequences=True))
    model.add(LSTM(80))
    model.add(Dense(80,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [37]:
model = create_model(vocab_size+1,seq_ln)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 10, 10)            134670    
_________________________________________________________________
lstm_11 (LSTM)               (None, 10, 80)            29120     
_________________________________________________________________
lstm_12 (LSTM)               (None, 80)                51520     
_________________________________________________________________
dense_14 (Dense)             (None, 80)                6480      
_________________________________________________________________
dense_15 (Dense)             (None, 13467)             1090827   
Total params: 1,312,617
Trainable params: 1,312,617
Non-trainable params: 0
_________________________________________________________________


In [48]:
#Fitting model on our data
#Batch_size = 64 is just an arbitory number
#Training upto 55 epochs
#verbose = 2 (has values 0,1,2 for which type of summary you want while training your model)
model.fit(X,y,batch_size=64,epochs=55,verbose=2)

Epoch 1/55
 - 333s - loss: 4.6320 - acc: 0.1887
Epoch 2/55
 - 348s - loss: 4.5802 - acc: 0.1909
Epoch 3/55
 - 353s - loss: 4.5332 - acc: 0.1935
Epoch 4/55
 - 355s - loss: 4.4889 - acc: 0.1960
Epoch 5/55
 - 339s - loss: 4.4488 - acc: 0.1981
Epoch 6/55
 - 346s - loss: 4.4106 - acc: 0.2006
Epoch 7/55
 - 355s - loss: 4.3765 - acc: 0.2028
Epoch 8/55
 - 356s - loss: 4.3434 - acc: 0.2056
Epoch 9/55
 - 355s - loss: 4.3147 - acc: 0.2074
Epoch 10/55
 - 352s - loss: 4.2857 - acc: 0.2089
Epoch 11/55
 - 347s - loss: 4.2587 - acc: 0.2115
Epoch 12/55
 - 347s - loss: 4.2312 - acc: 0.2135
Epoch 13/55
 - 354s - loss: 4.2069 - acc: 0.2155
Epoch 14/55
 - 356s - loss: 4.1836 - acc: 0.2169
Epoch 15/55
 - 359s - loss: 4.1617 - acc: 0.2192
Epoch 16/55
 - 363s - loss: 4.1397 - acc: 0.2211
Epoch 17/55
 - 360s - loss: 4.1189 - acc: 0.2226
Epoch 18/55
 - 344s - loss: 4.0999 - acc: 0.2241
Epoch 19/55
 - 345s - loss: 4.0801 - acc: 0.2259
Epoch 20/55
 - 333s - loss: 4.0629 - acc: 0.2277
Epoch 21/55
 - 328s - loss: 4

In [49]:
from keras.preprocessing.sequence import pad_sequences

#Using seed text to generate new text sequence
#Genrating words equal to "num_of_gen_words"
#using pad_seqences for cases where our seed text does not match our seed length of 10 words 
#pad_sequences truncates the word according to "pre" or "post" passed to it
#Getting the word form tokenizer using the predicied word index
#joining whole output and returning it

def generated_text(model,tokenizer,seq_ln,seed_text,num_of_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_of_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_ln,truncating='pre')
        predicted_wrd_index = model.predict_classes(pad_encoded,verbose=0)[0]
        predicted_wrd = tokenizer.index_word[predicted_wrd_index]
        input_text += ' '+predicted_wrd
        output_text.append(predicted_wrd)
        
    return ' '.join(output_text)

In [78]:
# You can give a random seed text from training file itself like --> ' '.join(text_seq[70])
#Or give custom seed text like -->
seed_text = "Let me begin by wishing ‘Namaskar’ to all of you"

In [80]:
gen_text = generated_text(model,tokenizer,seq_ln,seed_text,num_of_gen_words=100)

In [81]:
print(f"Seed text : --- > {seed_text}")
print("\n\n<<--------------------------------------->>\n\n")
print(f"Genrerated text : --- > {gen_text}")

Seed text : --- > Let me begin by wishing ‘Namaskar’ to all of you


<<--------------------------------------->>


Genrerated text : --- > and thank you for this honour and you will have observed the starting effective role in the world of nuclear and agri vessels we have to do that in the world we have decided to do so as well as a great persona of the world we have a chance to be a very productive forum i am happy to be here to do so as well as a great proof of the world ’s largest and spiritual ranking in the world bank imf is a part of the world we have a new era of the world and the
